In [ ]:
import numpy as np
import pandas as pd
import pandas_ta as ta

from plotly import graph_objects as go
from tqdm import tqdm

In [ ]:
xau = pd.read_csv("data/xauusd_1h.csv", index_col="datetime", parse_dates=['datetime'])
xau.head()

In [ ]:
xau.index[0]

In [ ]:
def pivot_high(highs:pd.Series, window:int=14):
    len = highs.size
    ph = np.zeros(shape=(len, ), dtype=float)
    
    for i in range(window, len-window):
        if highs.iloc[i] == highs.iloc[i-window:i+window].max():
            ph[i] = highs.iloc[i]
    return ph

def pivot_low(lows:pd.Series, window:int=14):
    len = lows.size
    pl = np.zeros(shape=(len, ), dtype=float)
    
    for i in range(window, len-window):
        if lows.iloc[i] == lows.iloc[i-window:i+window].min():
            pl[i] = lows.iloc[i]
    return pl

In [ ]:
def pivots(highs:pd.Series, lows:pd.Series, window:int) -> pd.Series:
    len = highs.size
    if lows.size != len:
        raise BufferError("`highs` and `lows` must have the same size!")
    pivots = pd.Series(data=0, index=highs.index)
    last_pivot = -1
    last_pivot_index = 0
    for i in tqdm(range(window, len-window)):
        if highs.iloc[i] == highs.iloc[i-window:i+window].max():
            if last_pivot == 1 and highs.iloc[last_pivot_index] < highs.iloc[i]:     # Previous pivot is a lower pivot high?!
                pivots.iloc[last_pivot_index] = 0       # Invalidate the previous pivot high
                last_pivot_index = i                    # Set new pivot high
                pivots.iloc[last_pivot_index] = last_pivot
            elif last_pivot == -1:  # Previous pivot is a pivot low?!
                last_pivot_index = i            # Set pivot high
                last_pivot = 1
                pivots.iloc[last_pivot_index] = last_pivot
        elif lows.iloc[i] == lows.iloc[i-window:i+window].min():
            if last_pivot == -1 and lows.iloc[last_pivot_index] > lows.iloc[i]:    # Previous pivot is a higher pivot low?!
                pivots.iloc[last_pivot_index] = 0       # Invalidate the previous pivot low
                last_pivot_index = i                    # Set new pivot low
                pivots.iloc[last_pivot_index] = last_pivot
            elif last_pivot == 1:       # Previous pivot is a pivot high
                last_pivot_index = i            # Set pivot low
                last_pivot = -1
                pivots.iloc[last_pivot_index] = last_pivot
    return pivots

In [ ]:
xau['pivots'] = pivots(xau['high'], xau['low'], window=7)

In [ ]:
xau[xau['pivots'] == -1]['pivots'].size

In [ ]:
def plot_data(df):
    fig = go.Figure(
        data=[go.Candlestick(x=df.index,
                            close=df['close'],
                            open=df['open'],
                            low=df['low'],
                            high=df['high'])]
    )

    fig.update_layout(title="XAUUSD - H1",
                    xaxis_title="Date Time",
                    yaxis_title="Price",
                    xaxis=dict(type="category"))

    fig.show()

In [ ]:
where = np.where(xau['pivots'] == 1)[0]
print(where)

In [ ]:
plot_data(xau.iloc[where[5]-50:where[5]+50])

In [ ]:
def head_and_shoulders(df:pd.DataFrame) -> pd.Series:
    if ['pivots', 'open', 'close'] not in df.columns:
        raise ValueError("The columns `['pivots', 'open', 'close']` must exist...")

    buy_sell = pd.Series(data=0, index=df.index)
    len = df.size
    for i in range(len):
        pass
    
    return buy_sell